In [3]:
import sqlite3
import pandas as pd
import numpy as np
import mlflow
from datetime import datetime, timedelta
from alibi_detect.cd import TabularDrift

ModuleNotFoundError: No module named 'alibi_detect'

In [ ]:
# 📅 Define período de referência (dados antigos) e análise (últimos 30 dias)
hoje = datetime.today()
limite = hoje - timedelta(days=30)

In [ ]:
# 📁 Conecta ao banco SQLite
DB_PATH = "app/data/extraidos/dados.db"
conn = sqlite3.connect(DB_PATH)

# 🔍 Consulta os dados
query = """
SELECT 
    formacao_academica,
    informacoes_profissionais_conhecimentos_tecnicos,
    cv_pt,
    data_candidatura
FROM prospects
WHERE data_candidatura IS NOT NULL
"""
df = pd.read_sql_query(query, conn)
conn.close()

In [ ]:
# 🧼 Prepara os dados
df["data_candidatura"] = pd.to_datetime(df["data_candidatura"], errors="coerce")
df = df.dropna(subset=["data_candidatura"])

# 📊 Divide os dados
df_ref = df[df["data_candidatura"] < limite].drop(columns=["data_candidatura"])
df_cur = df[df["data_candidatura"] >= limite].drop(columns=["data_candidatura"])

In [ ]:
# 🔠 Codifica texto como números (necessário para Alibi Detect)
df_all = pd.concat([df_ref, df_cur])
df_encoded = df_all.apply(lambda col: pd.factorize(col)[0])

In [ ]:
# Divide novamente após codificação
n_ref = len(df_ref)
X_ref = df_encoded.iloc[:n_ref].to_numpy()
X_cur = df_encoded.iloc[n_ref:].to_numpy()

In [ ]:
# 🧪 Detector de drift tabular
cd = TabularDrift(x_ref=X_ref, p_val=0.05, categories_per_feature={0: None, 1: None, 2: None})
preds = cd.predict(X_cur)

# 📊 Resultados
drift_detectado = preds['data']['is_drift']
p_values = preds['data']['p_val']
drift_features = [df_encoded.columns[i] for i, p in enumerate(p_values) if p < 0.05]

In [ ]:
# 📦 Log no MLflow
timestamp = hoje.strftime("%Y%m%d_%H%M%S")
with mlflow.start_run(run_name=f"Drift_Alibi_{timestamp}"):
    mlflow.log_param("drift_detectado", drift_detectado)
    mlflow.log_param("features_com_drift", ", ".join(drift_features))
    mlflow.log_metric("num_features_drift", len(drift_features))

print(f"✅ Drift detectado: {drift_detectado}")
print(f"📊 Features com drift: {drift_features}")